### **_Importar librerias_**

In [48]:
# Principales
import pandas as pd
import numpy as np

# Modificaciones 
import re
from unidecode import unidecode
import unicodedata
import datetime


# visuzalizaciones
import missingno as msno
import matplotlib as plt 
import plotly.express as px 


In [49]:
pd.set_option("display.max_columns",None)

### **_Lectura de datos_**

In [51]:
datos_pisosmadrid_normal = pd.read_excel("./04-PisosMadrid/PisosMadrid.xlsx")
datos_pisosmadrid_detalles = pd.read_excel("./04-PisosMadrid/Detalle_pisosmadrid.xlsx")


In [52]:
dbcp_ordenado= pd.read_excel('../02-datos-limpios/distr_barrios_codpostal.xlsx')

In [53]:
datos_pisosmadrid_normal.head()

,Campo,Campo1,Precio,dormitorios,banos,m2construidos,Descripción,btn
0,https://www.pisosmadrid.com.es/propiedad/40866...,"Piso en venta en Ibiza, Madrid",520.000 €,3 Dormitorio(s),1 baño,70 m2,Precioso piso reformado de 60m2 enfrente del R...,Detalles
1,https://www.pisosmadrid.com.es/propiedad/40865...,Piso en venta en Madrid,329.000 €,3 Dormitorio(s),2 Baño(s),95 m2,~SERVICHECK INMOBILIARIA comercializa un preci...,Detalles
2,https://www.pisosmadrid.com.es/propiedad/40865...,"Piso en venta en Arganzuela, Madrid",740.000 €,4 Dormitorio(s),3 Baño(s),158 m2,Se ofrece una magnífica oportunidad para adqui...,Detalles
3,https://www.pisosmadrid.com.es/propiedad/40864...,"Piso en venta en Los Angeles, Madrid",70.256 €,3 Dormitorio(s),1 baño,57 m2,"Vivienda situada en Calle Godella,, en Madrid ...",Detalles
4,https://www.pisosmadrid.com.es/propiedad/40864...,Piso en venta en Madrid,97.920 €,2 Dormitorio(s),1 baño,50 m2,"Vivienda situada en Calle Transmonte,, en Madr...",Detalles


In [54]:
# Bien aqui vamos a construir una funcion que recorra todo el dataframe y tome los caracteres especiales 
# las comas, los puntos, entre otros y los reemplaza con nada o sea basicamente eliminarlos. 
#Silvia la aplicacion de la funcion a  todo el dataframe puede provocar que numeros tomen valores incorrectos
def limpiar_texto(x):
   return re.sub(r'[^\w\s]','',unidecode(str(x)))

#datos_pisosmadrid_normal = datos_pisosmadrid_normal.apply(lambda col: col.apply(limpiar_texto)if col.name != "Campo"else col)

In [55]:
#  Aqui creamos una funcion que recorrera la columna que especificamos quitanto el valor que nosotoros le digamos y sin distincion 
# alguna sobre si es mayuscula o minuscula.                                                                                 
def eliminar_palabra(palabra):
    return re.sub(r'(?i)\bdormitorios\b', '', palabra)

datos_pisosmadrid_normal["dormitorios"]= datos_pisosmadrid_normal["dormitorios"].apply(eliminar_palabra)

# --------------------------------------------------------------------------

def eliminar_palabra(palabra):
    return re.sub(r'(?i)\bbano\b', '', palabra)

datos_pisosmadrid_normal["banos"]= datos_pisosmadrid_normal["banos"].apply(eliminar_palabra)

# --------------------------------------------------------------------------
def eliminar_palabra(palabra):
    return re.sub(r'(?i)\bbanos\b', '', palabra)

datos_pisosmadrid_normal["banos"]= datos_pisosmadrid_normal["banos"].apply(eliminar_palabra)
# --------------------------------------------------------------------------

def eliminar_palabra(palabra):
    return re.sub(r'(?i)\bm2\b', '', palabra)

datos_pisosmadrid_normal["m2construidos"]= datos_pisosmadrid_normal["m2construidos"].apply(eliminar_palabra)
# --------------------------------------------------------------------------

def eliminar_palabra(palabra):
    return re.sub(r'(?i)\bEUR\b', '', palabra)

datos_pisosmadrid_normal["Precio"]= datos_pisosmadrid_normal["Precio"].apply(eliminar_palabra)

In [56]:
# silvia añado el tipo_lugar y elimino los que no se van a analizar y campo referencia para el merge

In [57]:
# Función para extraer números entre '/' y '-'
def extraer_numeros(texto):
    patron = r"/(\d+)-"
    resultado = re.search(patron, texto)
    return resultado.group(1) if resultado else None

# Aplicar la función a la columna 'campo'
datos_pisosmadrid_normal["referencia"]=  datos_pisosmadrid_normal['Campo'].apply(extraer_numeros)


In [58]:

# Extraer la primera palabra de la columna 'titulo' 
datos_pisosmadrid_normal['tipo_lugar'] = datos_pisosmadrid_normal['Campo1'].apply(lambda x: x.split()[0]) 
#silvia añado la elimninacion de aquello que no son pisos
# Lista de tipos a eliminar
tipos_eliminar = ['Local', 'Garaje', 'Parking', 'Oficina', 'Terreno', 'Nave', 'Edificio', 'Solar', 'Parcela', 'Trastero']

# Filtrar el DataFrame para eliminar las filas con los valores especificados
datos_pisosmadrid_normal= datos_pisosmadrid_normal[~datos_pisosmadrid_normal['tipo_lugar'].isin(tipos_eliminar)]



In [59]:
# Aqiu se asignaron los nuesvos nombres por cada columna de forma indivual.
# El proceso tambien se puede hacer de forma mas corta.... Declaras una variable nueva con los nombres que quieres darles a las columnas
# en el orden que quieres poner los nombres y despues los asignas a las columnas.
datos_pisosmadrid_normal = datos_pisosmadrid_normal.rename(columns={"Campo":"url_website"})
 # --------------------------------------------------------------------------                  
datos_pisosmadrid_normal = datos_pisosmadrid_normal.rename(columns={"Campo1":"descripcion"})
 # --------------------------------------------------------------------------
                    
datos_pisosmadrid_normal = datos_pisosmadrid_normal.rename(columns={"Precio":"precio"})
  # --------------------------------------------------------------------------
                   
datos_pisosmadrid_normal = datos_pisosmadrid_normal.rename(columns={"dormitorios":"cant_dormitorios"})
 # --------------------------------------------------------------------------
                    
datos_pisosmadrid_normal = datos_pisosmadrid_normal.rename(columns={"banos":"cant_banos"})
# --------------------------------------------------------------------------

datos_pisosmadrid_normal = datos_pisosmadrid_normal.rename(columns={"m2construidos":"metros_cuadrados_construidos"})
  # --------------------------------------------------------------------------
                   
datos_pisosmadrid_normal = datos_pisosmadrid_normal.rename(columns={"Descripción":"descripcion_extendida"})
   # --------------------------------------------------------------------------
                  
datos_pisosmadrid_normal = datos_pisosmadrid_normal.drop(columns=["btn"])  

In [60]:
datos_pisosmadrid_normal.columns

Index(['url_website', 'descripcion', 'precio', 'cant_dormitorios',
       'cant_banos', 'metros_cuadrados_construidos', 'descripcion_extendida',
       'referencia', 'tipo_lugar'],
      dtype='object')

### **_Aqui se termina la parte de --pisosmadrid_normal-- y empieza la de --pisosmadrid_detalles--_**

In [61]:
datos_pisosmadrid_detalles["Campo18"].unique()

array(['Jardín:', nan], dtype=object)

In [62]:
datos_pisosmadrid_detalles["Campo12"].value_counts()

Campo12
0.0     327
3.0     319
2.0     272
1.0     109
4.0      98
5.0      34
6.0      12
7.0       7
8.0       5
9.0       2
10.0      1
20.0      1
Name: count, dtype: int64

In [63]:
datos_pisosmadrid_detalles["Campo1"].value_counts()

Campo1
0 Dormitorio(s)     327
3 Dormitorio(s)     319
2 Dormitorio(s)     272
1 dormitorio        109
4 Dormitorio(s)      98
5 Dormitorio(s)      34
6 Dormitorio(s)      12
7 Dormitorio(s)       7
8 Dormitorio(s)       5
9 Dormitorio(s)       2
10 Dormitorio(s)      1
20 Dormitorio(s)      1
Name: count, dtype: int64

In [64]:
datos_pisosmadrid_detalles["Campo14"].value_counts()

Campo14
1.0     416
2.0     350
0.0     266
3.0     108
4.0      25
5.0       9
6.0       6
7.0       3
31.0      1
10.0      1
11.0      1
8.0       1
Name: count, dtype: int64

In [65]:
datos_pisosmadrid_detalles["Campo2"].value_counts()

Campo2
1 baño        416
2 Baño(s)     350
0 Baño(s)     266
3 Baño(s)     108
4 Baño(s)      25
5 Baño(s)       9
6 Baño(s)       6
7 Baño(s)       3
31 Baño(s)      1
10 Baño(s)      1
11 Baño(s)      1
8 Baño(s)       1
Name: count, dtype: int64

In [66]:
datos_pisosmadrid_detalles["Campo3"].value_counts()

Campo3
70 m2      21
0 m2       18
22 m2      18
m2         16
80 m2      15
           ..
18 m2       1
314 m2      1
15 m2       1
208 m2      1
3995 m2     1
Name: count, Length: 305, dtype: int64

In [67]:
datos_pisosmadrid_detalles["Metros"].value_counts()

Metros
70.0      21
0.0       18
22.0      18
80.0      15
65.0      15
          ..
726.0      1
769.0      1
460.0      1
1600.0     1
3995.0     1
Name: count, Length: 304, dtype: int64

In [68]:
# Aqui se eliminaron una lista de columnas elegidas por mi. Se hizo un previo analisis del valor que proporcionaban cada una y la conclusion 
# fue que el valor que proprcionaban era igual o menor que nada.
datos_pisosmadrid_detalles = datos_pisosmadrid_detalles.drop(columns=["Campo4","Campo6","Campo7","Campo8","Campo9","Campo10","Campo15","Campo16","Campo17","Campo18","Provincia","Campo11",
                            "Campo12","Campo14", "Campo13","Metros"])

In [69]:
def limpiar_texto(x):
    return re.sub(r'[^\w\s]','',unidecode(str(x)))
datos_pisosmadrid_detalles = datos_pisosmadrid_detalles.apply(lambda col: col.apply(limpiar_texto)if col.name !="Publicado" else col)

In [70]:
# Aqui creamos una funcion que recorrera la columna que especificamos quitanto el valor que nosotoros le digamos y sin distincion 
# alguna sobre si es mayuscula o minuscula.
def eliminar_palabra(palabra):
    return re.sub(r'(?i)\bdormitorios\b', '', palabra)

datos_pisosmadrid_detalles["Campo1"]= datos_pisosmadrid_detalles["Campo1"].apply(eliminar_palabra)

# --------------------------------------------------------------------------

def eliminar_palabra(palabra):
    return re.sub(r'(?i)\bbano\b', '', palabra)

datos_pisosmadrid_detalles["Campo2"]= datos_pisosmadrid_detalles["Campo2"].apply(eliminar_palabra)

# --------------------------------------------------------------------------
def eliminar_palabra(palabra):
    return re.sub(r'(?i)\bbanos\b', '', palabra)

datos_pisosmadrid_detalles["Campo2"]= datos_pisosmadrid_detalles["Campo2"].apply(eliminar_palabra)
# --------------------------------------------------------------------------

def eliminar_palabra(palabra):
    return re.sub(r'(?i)\bm2\b', '', palabra)

datos_pisosmadrid_detalles["Campo3"]= datos_pisosmadrid_detalles["Campo3"].apply(eliminar_palabra)
# --------------------------------------------------------------------------

def eliminar_palabra(palabra):
    return re.sub(r'(?i)\bEUR\b', '', palabra)

datos_pisosmadrid_detalles["Texto"]= datos_pisosmadrid_detalles["Texto"].apply(eliminar_palabra)

In [71]:
datos_pisosmadrid_detalles.columns

Index(['Título', 'Campo1', 'Campo2', 'Campo3', 'Campo5', 'Tipo', 'Ciudad',
       'Area', 'Piscina', 'Garaje', 'Jardín', 'Campo19', 'Referencia',
       'Publicado', 'Visitas', 'Texto'],
      dtype='object')

In [72]:

nuevos_nombres = ["descripcion","cant_dormitorios","cant_banos","m_cuadrados","descripcion2","tipo_lugar","ciudad","barrio/localidad/distrito","piscina","garaje",
                  "jardin","descripcion3","referencia","fecha_publicacion","n.visitas","precio"]
datos_pisosmadrid_detalles.columns= nuevos_nombres

In [73]:
#silvia añado la elimninacion de aquello que no son pisos
# Lista de tipos a eliminar
tipos_eliminar = ['Local', 'Garaje', 'Parking', 'Oficina', 'Terreno', 'Nave', 'Edificio', 'Solar', 'Parcela', 'Trastero']

# Filtrar el DataFrame para eliminar las filas con los valores especificados
datos_pisosmadrid_detalles= datos_pisosmadrid_detalles[~datos_pisosmadrid_detalles['tipo_lugar'].isin(tipos_eliminar)]
#elimino PM de referencia
datos_pisosmadrid_detalles["referencia"] = datos_pisosmadrid_detalles["referencia"].str.replace("PM", "")

In [74]:
datos_pisosmadrid_detalles["referencia"].value_counts(dropna=False)

referencia
4086665    1
3974110    1
3978041    1
3977655    1
3975253    1
          ..
4049211    1
4049210    1
4048885    1
4047995    1
3863841    1
Name: count, Length: 883, dtype: int64

In [75]:
# Aqui vamos a convertir el tipo de dato que tiene una columna en especifico al tipo datetime. 
datos_pisosmadrid_detalles["fecha_publicacion"]=pd.to_datetime(datos_pisosmadrid_detalles["fecha_publicacion"], dayfirst=True)

In [76]:
# Aqui vamos a hacer una serie de reemplazamientos indivuales . 


datos_pisosmadrid_detalles["descripcion3"]= datos_pisosmadrid_detalles["descripcion3"].str.replace(r'Caracteristicas\n\t\t\t\t', "",regex=True)
datos_pisosmadrid_detalles["descripcion3"]= datos_pisosmadrid_detalles["descripcion3"].str.replace(r'Caracteristicas',"",regex=True).replace("",np.nan)

datos_pisosmadrid_detalles["descripcion2"]= datos_pisosmadrid_detalles["descripcion2"].str.replace(r'Descripcion\n\t\t\t\t', "",regex=True)
datos_pisosmadrid_detalles["descripcion2"]= datos_pisosmadrid_detalles["descripcion2"].str.replace(r'Descripcion\n', "",regex=True)


In [77]:
columnas_01= ["piscina","garaje","jardin"]

datos_pisosmadrid_detalles[columnas_01]=datos_pisosmadrid_detalles[columnas_01].replace({"Si":1,"No":0})

In [78]:
# Aqui vamos a crear una variabel que guarde un diccionario con las palabras que quremos tomar de la columna descripcion3 y el nombre
# de la columna a la que queremos que la palabra se vaya.
palabras_a_columnas = {
    "Calefaccion" : "calefaccion",
    "Trastero"    : "trastero",
    "Amueblado"   :"amueblado",
    "Chimenea"    :"chimenea",
    "Lavadero"    :"lavadero",
    "Videoportero":"videoportero",
    "Alarma"      :"alarma",
    "Aire acondicionado": "aire_acondicionado"
}

# Aqui creamos una variable donde guardaremos el nombre de la columna de donde se van a extraer todas esas palabras mencionadas antes.
columna_original = 'descripcion3'

# Aqui vamos a crear un bucle for que itera sobre las columnas y encuentra los valores que escribimos antes. 
for palabra, nueva_columna in palabras_a_columnas.items():
    # Usar apply para crear la columna con 1 si la palabra está presente, de lo contrario 0
    datos_pisosmadrid_detalles[nueva_columna] = datos_pisosmadrid_detalles[columna_original].apply(lambda x: 1 if isinstance(x, str) and palabra.lower() in x.lower() else 0)

# Este bucle itera sobre las claves del diccionario palabras_a_columnas. 

for palabra in palabras_a_columnas.keys():
    datos_pisosmadrid_detalles[columna_original] = datos_pisosmadrid_detalles[columna_original].str.replace(rf'\b{palabra}\b', '', regex=True)
datos_pisosmadrid_detalles[columna_original] = datos_pisosmadrid_detalles[columna_original].str.strip()
# Reemplazar valores vacíos o "nan" con NaN en la columna original
datos_pisosmadrid_detalles[columna_original] = datos_pisosmadrid_detalles[columna_original].replace(["", "nan"], np.nan)



C:\Users\extas\AppData\Local\Temp\ipykernel_1464\3063665124.py:28: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  datos_pisosmadrid_detalles[columna_original] = datos_pisosmadrid_detalles[columna_original].replace(["", "nan"], np.nan)


In [79]:
# Aqui eliminamos la columna de la que sacamos todas las palabras anteriores, ahora solo queda con un nulos asi que hay que dejarla ir.
datos_pisosmadrid_detalles= datos_pisosmadrid_detalles.drop("descripcion3",axis=1)

In [80]:
# datos_pisosmadrid_detalles = datos_pisosmadrid_detalles.to_csv("pisosmadrid_detalles_tratado.csv")

### **_EDA/ Exploratory Data Analysis y Preprocesamiento PisosMadrid normal y detalle_**

In [81]:
datos_pisosmadrid_detalles.shape

(883, 23)

In [82]:
datos_pisosmadrid_normal.shape

(893, 9)

In [83]:
datos_pisosmadrid_normal.head(3)

,url_website,descripcion,precio,cant_dormitorios,cant_banos,metros_cuadrados_construidos,descripcion_extendida,referencia,tipo_lugar
0,https://www.pisosmadrid.com.es/propiedad/40866...,"Piso en venta en Ibiza, Madrid",520.000 €,3 Dormitorio(s),1 baño,70,Precioso piso reformado de 60m2 enfrente del R...,4086665,Piso
1,https://www.pisosmadrid.com.es/propiedad/40865...,Piso en venta en Madrid,329.000 €,3 Dormitorio(s),2 Baño(s),95,~SERVICHECK INMOBILIARIA comercializa un preci...,4086544,Piso
2,https://www.pisosmadrid.com.es/propiedad/40865...,"Piso en venta en Arganzuela, Madrid",740.000 €,4 Dormitorio(s),3 Baño(s),158,Se ofrece una magnífica oportunidad para adqui...,4086503,Piso


In [84]:
datos_pisosmadrid_detalles.head(3)

,descripcion,cant_dormitorios,cant_banos,m_cuadrados,descripcion2,tipo_lugar,ciudad,barrio/localidad/distrito,piscina,garaje,jardin,referencia,fecha_publicacion,n.visitas,precio,calefaccion,trastero,amueblado,chimenea,lavadero,videoportero,alarma,aire_acondicionado
0,Piso en venta en Ibiza Madrid,3,1,70,Precioso piso ...,Piso,Madrid,Ibiza,0,0,0,4086665,2024-10-24,40,520000,1,0,0,0,0,0,0,0
1,Piso en venta en Madrid,3,2,95,SERVICHECK INMOBILIARIA comercializa un precio...,Piso,Madrid,nan,0,1,1,4086544,2024-10-24,20,329000,1,1,1,0,0,1,0,0
2,Piso en venta en Arganzuela Madrid,4,3,158,Se ofrece una magnifica oportunidad para adqui...,Piso,Madrid,Arganzuela,0,0,0,4086503,2024-10-24,30,740000,1,0,0,0,0,1,1,0


In [85]:
datos_pisosmadrid_detalles= datos_pisosmadrid_detalles.rename(columns={"m_cuadrados":"metros_cuadrados_construidos"})
datos_pisosmadrid_detalles= datos_pisosmadrid_detalles.rename(columns={"descripcion2":"descripcion_extendida"})

In [86]:
datos_pisosmadrid_detalles["fecha_publicacion"]=pd.to_datetime(datos_pisosmadrid_detalles["fecha_publicacion"],format='%Y-%m-%d',errors="coerce")

In [87]:
# Silvia , esto no se puede asignar directamente

#datos_pisosmadrid_detalles["url_website"] = datos_pisosmadrid_normal["url_website"]

In [88]:
# Reemplazamiento de la palabra dormitorio 
datos_pisosmadrid_detalles['cant_dormitorios'] = datos_pisosmadrid_detalles['cant_dormitorios'] \
    .replace(r"\bdormitorio\b\s*", "", regex=True)
datos_pisosmadrid_normal['cant_dormitorios'] = datos_pisosmadrid_normal['cant_dormitorios'] \
    .replace(r"\bdormitorio\b\s*", "", regex=True)

In [89]:
datos_pisosmadrid_detalles.shape

(883, 23)

In [90]:
col_conversion= ["piscina","garaje","jardin","n.visitas","cant_dormitorios",
                 "precio","cant_banos","metros_cuadrados_construidos" ]
# Reemplazar 'nan' y cadenas vacías '' por 0 en todas las columnas relevantes
datos_pisosmadrid_detalles[col_conversion] = datos_pisosmadrid_detalles[col_conversion].replace({'nan': '0', '': '0'})

# Convertir las columnas a tipo numérico (float primero para manejar correctamente los valores)
datos_pisosmadrid_detalles[col_conversion] = datos_pisosmadrid_detalles[col_conversion].apply(pd.to_numeric, errors='coerce')

# Convertir las columnas a enteros, llenando NaN con 0 antes de la conversión
datos_pisosmadrid_detalles[col_conversion] = datos_pisosmadrid_detalles[col_conversion].fillna(0).astype("int64")


In [91]:
datos_pisosmadrid_detalles.shape

(883, 23)

In [92]:
datos_pisosmadrid_detalles.dtypes

descripcion                             object
cant_dormitorios                         int64
cant_banos                               int64
metros_cuadrados_construidos             int64
descripcion_extendida                   object
tipo_lugar                              object
ciudad                                  object
barrio/localidad/distrito               object
piscina                                  int64
garaje                                   int64
jardin                                   int64
referencia                              object
fecha_publicacion               datetime64[ns]
n.visitas                                int64
precio                                   int64
calefaccion                              int64
trastero                                 int64
amueblado                                int64
chimenea                                 int64
lavadero                                 int64
videoportero                             int64
alarma       

In [93]:
datos_pisosmadrid_detalles.isna().sum()

descripcion                     0
cant_dormitorios                0
cant_banos                      0
metros_cuadrados_construidos    0
descripcion_extendida           0
tipo_lugar                      0
ciudad                          0
barrio/localidad/distrito       0
piscina                         0
garaje                          0
jardin                          0
referencia                      0
fecha_publicacion               1
n.visitas                       0
precio                          0
calefaccion                     0
trastero                        0
amueblado                       0
chimenea                        0
lavadero                        0
videoportero                    0
alarma                          0
aire_acondicionado              0
dtype: int64

In [94]:
datos_pisosmadrid_detalles["piscina"].unique()

array([0, 1], dtype=int64)

In [95]:
datos_pisosmadrid_detalles["garaje"].unique()

array([0, 1], dtype=int64)

In [96]:
datos_pisosmadrid_detalles["jardin"].unique()

array([0, 1], dtype=int64)

In [97]:
datos_pisosmadrid_detalles["n.visitas"].unique()

array([ 40,  20,  30,  60,  50,   0,  10,  70,  80, 170, 130,  90, 110,
       100, 120, 190, 150, 160, 140, 180, 200], dtype=int64)

In [98]:
datos_pisosmadrid_detalles["precio"].unique()

array([  520000,   329000,   740000,    70256,    97920,    69313,
        1400000,  1800000,   380000,   235000,  1354900,  1129000,
         320000,  2099000,   960000,   315000,   220000,   299000,
        2350000,   515000,   870000,   625000,   250000,   258000,
         298500,   322900,   317000,   322000,  1500000,   189900,
         575000,   675000,   275000,   370000,   285000,   475000,
         570000,  1300000,   249000,   460000,  1355000,    54000,
         539000,   600000,   135000,   843800,   970000,   850000,
        1770000,   655000,   301000,  1150000,   800000,   239000,
         238000,   210000,   335000,  1550000,  1450000,   500000,
         240000,  2475000,   925000,  1140000,  1330000,   650000,
         190000,   173000,   266500,   230000,   286700,   459000,
         179990,   400000,   430000,   185000,   145000,   595000,
         165000,   260000,  1975000,  1295000,   859000,  1359000,
        1149000,  1039000,   995000,  1195000,   820000,   975

In [99]:
columnas_texto = datos_pisosmadrid_detalles.select_dtypes(include=['object']).columns

# Paso 2: Limpiar las columnas de texto aplicando .str.strip(), .lower(), etc.
for col in columnas_texto:
    # Reemplazar NaN por cadena vacía
    datos_pisosmadrid_detalles[col] = datos_pisosmadrid_detalles[col].fillna('')  # Reemplaza NaN por cadena vacía
    # Convertir a tipo str si es necesario
    datos_pisosmadrid_detalles[col] = datos_pisosmadrid_detalles[col].astype(str)
    # Limpiar espacios al principio y al final
    datos_pisosmadrid_detalles[col] = datos_pisosmadrid_detalles[col].str.strip()

In [100]:
columnas_comparar = ['descripcion', 'precio',
                     "cant_dormitorios","cant_banos","metros_cuadrados_construidos",
                     "descripcion_extendida"]

# Limitar ambos DataFrames a las primeras 1188 filas
datos_subset = datos_pisosmadrid_detalles[1:15]
datos2_subset = datos_pisosmadrid_normal[1:15]

# Comparar todas las columnas seleccionadas
coinciden = datos_subset[columnas_comparar] == (datos2_subset[columnas_comparar])

coinciden

,descripcion,precio,cant_dormitorios,cant_banos,metros_cuadrados_construidos,descripcion_extendida
1,True,False,False,False,False,False
2,False,False,False,False,False,False
3,False,False,False,False,False,False
4,True,False,False,False,False,False
5,False,False,False,False,False,False
7,False,False,False,False,False,False
8,False,False,False,False,False,False
16,False,False,False,False,False,False
17,False,False,False,False,False,False
18,False,False,False,False,False,False


In [101]:
datos_pisosmadrid_detalles["barrio/localidad/distrito"].value_counts()

barrio/localidad/distrito
nan                        120
Centro                      58
Salamanca                   38
Malasana                    37
Goya                        27
                          ... 
Cortes                       1
Alameda de Osuna             1
Valverde                     1
Ensanche Vallecas            1
Montecarmelo  El Goloso      1
Name: count, Length: 148, dtype: int64

In [102]:
datos_pisosmadrid_detalles["descripcion_extendida"].to_string()

'0       Precioso piso reformado de 60m2 enfrente del R...\n1       SERVICHECK INMOBILIARIA comercializa un precio...\n2       Se ofrece una magnifica oportunidad para adqui...\n3       Vivienda situada en Calle Godella en Madrid ca...\n4       Vivienda situada en Calle Transmonte en Madrid...\n5       Vivienda situada en Calle Callejo en Madrid ca...\n7       Gran Oportunidad en Malasana Vivienda amplia y...\n8       Que interesante propiedad El piso que describe...\n16      Inmobiliarias Encuentro VENDE este VIVIENDA CO...\n17      DP2020 vende fantastico piso ubicado en la zon...\n18      Este espectacular piso exterior en la calle Jo...\n19      Nos complace presentar esta espectacular vivie...\n20      WeGet Inmobiliaria VENDE acogedora vivienda en...\n21      Ubicada en el prestigioso barrio de Salamanca ...\n22      Remax Activo Residencial ofrece este fantastic...\n23      Te presentamos este fantastico estudio en la c...\n24      SERVIFINCAS Arganzuela te presenta en exclusiv.

In [103]:
datos_pisosmadrid_normal.isna().sum()

url_website                     0
descripcion                     0
precio                          0
cant_dormitorios                0
cant_banos                      0
metros_cuadrados_construidos    0
descripcion_extendida           0
referencia                      0
tipo_lugar                      0
dtype: int64

In [104]:
datos_pisosmadrid_detalles.isnull().sum()

descripcion                     0
cant_dormitorios                0
cant_banos                      0
metros_cuadrados_construidos    0
descripcion_extendida           0
tipo_lugar                      0
ciudad                          0
barrio/localidad/distrito       0
piscina                         0
garaje                          0
jardin                          0
referencia                      0
fecha_publicacion               1
n.visitas                       0
precio                          0
calefaccion                     0
trastero                        0
amueblado                       0
chimenea                        0
lavadero                        0
videoportero                    0
alarma                          0
aire_acondicionado              0
dtype: int64

In [105]:
dbcp_ordenado

,cod_distrito,nombre_distrito,cod_barrio,cod_barrio_2,nombre_barrio,codigo_Postal
0,1,Centro,11,1,Palacio,28013
1,1,Centro,12,2,Embajadores,28012
2,1,Centro,13,3,Cortes,28014
3,1,Centro,14,4,Justicia,28004
4,1,Centro,15,5,Universidad,28015
...,...,...,...,...,...,...
126,20,San Blas-Canillejas,208,8,Salvador,28037
127,21,Barajas,211,1,Alameda de Osuna,28042
128,21,Barajas,212,2,Aeropuerto,28042
129,21,Barajas,213,3,Corralejos,28042


In [106]:
datos_pisosmadrid_detalles.head()

,descripcion,cant_dormitorios,cant_banos,metros_cuadrados_construidos,descripcion_extendida,tipo_lugar,ciudad,barrio/localidad/distrito,piscina,garaje,jardin,referencia,fecha_publicacion,n.visitas,precio,calefaccion,trastero,amueblado,chimenea,lavadero,videoportero,alarma,aire_acondicionado
0,Piso en venta en Ibiza Madrid,3,1,70,Precioso piso reformado de 60m2 enfrente del R...,Piso,Madrid,Ibiza,0,0,0,4086665,2024-10-24,40,520000,1,0,0,0,0,0,0,0
1,Piso en venta en Madrid,3,2,95,SERVICHECK INMOBILIARIA comercializa un precio...,Piso,Madrid,nan,0,1,1,4086544,2024-10-24,20,329000,1,1,1,0,0,1,0,0
2,Piso en venta en Arganzuela Madrid,4,3,158,Se ofrece una magnifica oportunidad para adqui...,Piso,Madrid,Arganzuela,0,0,0,4086503,2024-10-24,30,740000,1,0,0,0,0,1,1,0
3,Piso en venta en Los Angeles Madrid,3,1,57,Vivienda situada en Calle Godella en Madrid ca...,Piso,Madrid,Los Angeles,0,0,0,4086495,2024-10-24,40,70256,0,0,0,0,0,0,0,0
4,Piso en venta en Madrid,2,1,50,Vivienda situada en Calle Transmonte en Madrid...,Piso,Madrid,nan,0,0,0,4086493,2024-10-24,30,97920,0,0,0,0,0,0,0,0


In [107]:
datos_pisosmadrid_detalles.shape

(883, 23)

In [108]:
datos_pisosmadrid_detalles["area"]=datos_pisosmadrid_detalles["barrio/localidad/distrito"]

In [109]:
# Aqui guardamos el nombre de la columna objetivo en una variable
columna_a_clasificar= "barrio/localidad/distrito"

# Creamos la funcion que clasificara nombres en las columnas nuevas y tomara los valores de la columna target
def clasificar_barrio_distrito(datos_pisosmadrid_detalles, dbcp_ordenado,columna_a_clasificar):
    distritos= dbcp_ordenado["nombre_distrito"].tolist()
    barrios=   dbcp_ordenado["nombre_barrio"].tolist()

    # Creamos las nuevas columnas y las llenamos de nulos 
    datos_pisosmadrid_detalles["barrios"]   =np.nan 
    datos_pisosmadrid_detalles["distrito"]  =np.nan 
    datos_pisosmadrid_detalles["localidad"] =np.nan 

    for idx, valor in datos_pisosmadrid_detalles[columna_a_clasificar].items():
        if valor in distritos:
            datos_pisosmadrid_detalles.at[idx,"distrito" ]= valor
        elif valor in barrios:
            datos_pisosmadrid_detalles.at[idx,"barrios"   ]= valor 
        else:
            datos_pisosmadrid_detalles.at[idx,"localidad"]=valor
    datos_pisosmadrid_detalles[columna_a_clasificar]= np.nan

clasificar_barrio_distrito(datos_pisosmadrid_detalles = datos_pisosmadrid_detalles,dbcp_ordenado=dbcp_ordenado,columna_a_clasificar="barrio/localidad/distrito")


C:\Users\extas\AppData\Local\Temp\ipykernel_1464\2805591847.py:18: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Ibiza' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  datos_pisosmadrid_detalles.at[idx,"barrios"   ]= valor
C:\Users\extas\AppData\Local\Temp\ipykernel_1464\2805591847.py:20: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'nan' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  datos_pisosmadrid_detalles.at[idx,"localidad"]=valor
C:\Users\extas\AppData\Local\Temp\ipykernel_1464\2805591847.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Arganzuela' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  datos_p

In [110]:
datos_pisosmadrid_detalles

,descripcion,cant_dormitorios,cant_banos,metros_cuadrados_construidos,descripcion_extendida,tipo_lugar,ciudad,barrio/localidad/distrito,piscina,garaje,jardin,referencia,fecha_publicacion,n.visitas,precio,calefaccion,trastero,amueblado,chimenea,lavadero,videoportero,alarma,aire_acondicionado,area,barrios,distrito,localidad
0,Piso en venta en Ibiza Madrid,3,1,70,Precioso piso reformado de 60m2 enfrente del R...,Piso,Madrid,NaN,0,0,0,4086665,2024-10-24,40,520000,1,0,0,0,0,0,0,0,Ibiza,Ibiza,NaN,NaN
1,Piso en venta en Madrid,3,2,95,SERVICHECK INMOBILIARIA comercializa un precio...,Piso,Madrid,NaN,0,1,1,4086544,2024-10-24,20,329000,1,1,1,0,0,1,0,0,nan,NaN,NaN,nan
2,Piso en venta en Arganzuela Madrid,4,3,158,Se ofrece una magnifica oportunidad para adqui...,Piso,Madrid,NaN,0,0,0,4086503,2024-10-24,30,740000,1,0,0,0,0,1,1,0,Arganzuela,NaN,Arganzuela,NaN
3,Piso en venta en Los Angeles Madrid,3,1,57,Vivienda situada en Calle Godella en Madrid ca...,Piso,Madrid,NaN,0,0,0,4086495,2024-10-24,40,70256,0,0,0,0,0,0,0,0,Los Angeles,NaN,NaN,Los Angeles
4,Piso en venta en Madrid,2,1,50,Vivienda situada en Calle Transmonte en Madrid...,Piso,Madrid,NaN,0,0,0,4086493,2024-10-24,30,97920,0,0,0,0,0,0,0,0,nan,NaN,NaN,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1178,Piso en venta en Pueblo Nuevo Madrid,1,1,75,104\nExcepcional local bajo tipo loft transfor...,Piso,Madrid,NaN,0,0,0,3865903,2024-05-18,110,185000,0,0,0,0,0,0,0,0,Pueblo Nuevo,Pueblo Nuevo,NaN,NaN
1179,Apartamento en venta en Ibiza Madrid,3,2,173,Descubre este impresionante piso completamente...,Apartamento,Madrid,NaN,0,0,0,3866605,2024-05-19,120,1329000,0,0,0,0,0,1,0,0,Ibiza,Ibiza,NaN,NaN
1180,Piso en venta en Malasana Madrid,5,3,170,104\nIncreible oportunidad de inversion en el ...,Piso,Madrid,NaN,0,0,0,3865902,2024-05-18,200,760000,0,0,0,0,0,0,0,0,Malasana,NaN,NaN,Malasana
1182,Piso en venta en Montecarmelo El Goloso Madrid,4,2,142,600\nAprovecha esta oportunidad de compra de e...,Piso,Madrid,NaN,1,1,1,3865872,2024-05-18,150,825000,1,1,0,0,0,0,0,0,Montecarmelo El Goloso,NaN,NaN,Montecarmelo El Goloso


In [111]:
datos_pisosmadrid_detalles["distrito"].isna().sum()

707

In [112]:
datos_pisosmadrid_detalles["barrios"].isna().sum()

556

In [113]:
# Esta funcion rellena los distritos conociendo los barrios


def rellenar_distritos(datos_pisosmadrid_detalles,dbcp_ordenado):

    barrio_a_distrito=dict(zip(dbcp_ordenado["nombre_barrio"],dbcp_ordenado["nombre_distrito"]))

    for idx, fila in datos_pisosmadrid_detalles.iterrows():
        if pd.isna(fila["distrito"])and not pd.isna(fila["barrios"]):
            distrito = barrio_a_distrito.get(fila["barrios"])
            if distrito:
                datos_pisosmadrid_detalles.at[idx,"distrito"] = distrito 


rellenar_distritos(datos_pisosmadrid_detalles=datos_pisosmadrid_detalles,dbcp_ordenado=dbcp_ordenado)

In [114]:
datos_pisosmadrid_normal.shape, datos_pisosmadrid_detalles.shape

((893, 9), (883, 27))

In [115]:
datos_pisosmadrid_detalles["descripcion"]

0                         Piso en venta en Ibiza Madrid
1                               Piso en venta en Madrid
2                    Piso en venta en Arganzuela Madrid
3                   Piso en venta en Los Angeles Madrid
4                               Piso en venta en Madrid
                             ...                       
1178               Piso en venta en Pueblo Nuevo Madrid
1179               Apartamento en venta en Ibiza Madrid
1180                   Piso en venta en Malasana Madrid
1182    Piso en venta en Montecarmelo  El Goloso Madrid
1185                            Piso en venta en Madrid
Name: descripcion, Length: 883, dtype: object

In [116]:
columnas_eliminar=["descripcion","descripcion_extendida","precio","cant_dormitorios","cant_banos","metros_cuadrados_construidos","tipo_lugar"]
datos_pisosmadrid_detalles=datos_pisosmadrid_detalles.drop(columns=columnas_eliminar)


In [117]:
datos_pisosmadrid_detalles.reset_index(drop=True)
datos_pisosmadrid_normal.reset_index(drop=True)

,url_website,descripcion,precio,cant_dormitorios,cant_banos,metros_cuadrados_construidos,descripcion_extendida,referencia,tipo_lugar
0,https://www.pisosmadrid.com.es/propiedad/40866...,"Piso en venta en Ibiza, Madrid",520.000 €,3 Dormitorio(s),1 baño,70,Precioso piso reformado de 60m2 enfrente del R...,4086665,Piso
1,https://www.pisosmadrid.com.es/propiedad/40865...,Piso en venta en Madrid,329.000 €,3 Dormitorio(s),2 Baño(s),95,~SERVICHECK INMOBILIARIA comercializa un preci...,4086544,Piso
2,https://www.pisosmadrid.com.es/propiedad/40865...,"Piso en venta en Arganzuela, Madrid",740.000 €,4 Dormitorio(s),3 Baño(s),158,Se ofrece una magnífica oportunidad para adqui...,4086503,Piso
3,https://www.pisosmadrid.com.es/propiedad/40864...,"Piso en venta en Los Angeles, Madrid",70.256 €,3 Dormitorio(s),1 baño,57,"Vivienda situada en Calle Godella,, en Madrid ...",4086495,Piso
4,https://www.pisosmadrid.com.es/propiedad/40864...,Piso en venta en Madrid,97.920 €,2 Dormitorio(s),1 baño,50,"Vivienda situada en Calle Transmonte,, en Madr...",4086493,Piso
...,...,...,...,...,...,...,...,...,...
888,https://www.pisosmadrid.com.es/propiedad/38659...,"Piso en venta en Pueblo Nuevo, Madrid",185.000 €,1,1 baño,75,"104<br />\nExcepcional local bajo tipo loft, t...",3865903,Piso
889,https://www.pisosmadrid.com.es/propiedad/38666...,"Apartamento en venta en Ibiza, Madrid",1.329.000 €,3 Dormitorio(s),2 Baño(s),173,"Descubre este impresionante piso, completament...",3866605,Apartamento
890,https://www.pisosmadrid.com.es/propiedad/38659...,"Piso en venta en Malasaña, Madrid",760.000 €,5 Dormitorio(s),3 Baño(s),170,104<br />\n¡Increíble oportunidad de inversión...,3865902,Piso
891,https://www.pisosmadrid.com.es/propiedad/38658...,"Piso en venta en Montecarmelo - El Goloso, Madrid",825.000 €,4 Dormitorio(s),2 Baño(s),142,600<br />\nAprovecha esta oportunidad de compr...,3865872,Piso


In [118]:
# merge de pisosmadrid y redpiso 
dfpisosmadrid= pd.merge(datos_pisosmadrid_normal,datos_pisosmadrid_detalles,on=["referencia"],how="outer")

In [119]:
dfpisosmadrid.reset_index(drop=True)

,url_website,descripcion,precio,cant_dormitorios,cant_banos,metros_cuadrados_construidos,descripcion_extendida,referencia,tipo_lugar,ciudad,barrio/localidad/distrito,piscina,garaje,jardin,fecha_publicacion,n.visitas,calefaccion,trastero,amueblado,chimenea,lavadero,videoportero,alarma,aire_acondicionado,area,barrios,distrito,localidad
0,https://www.pisosmadrid.com.es/propiedad/38638...,Piso en venta en Madrid,60.000 €,3 Dormitorio(s),1 baño,54,PISO EN MADRID. ~Vivienda situada en Avenida P...,3863841,Piso,Madrid,NaN,0.0,0.0,0.0,2024-05-17,120.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,nan,NaN,NaN,nan
1,https://www.pisosmadrid.com.es/propiedad/38658...,"Piso en venta en Montecarmelo - El Goloso, Madrid",825.000 €,4 Dormitorio(s),2 Baño(s),142,600<br />\nAprovecha esta oportunidad de compr...,3865872,Piso,Madrid,NaN,1.0,1.0,1.0,2024-05-18,150.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,Montecarmelo El Goloso,NaN,NaN,Montecarmelo El Goloso
2,https://www.pisosmadrid.com.es/propiedad/38659...,"Piso en venta en Malasaña, Madrid",760.000 €,5 Dormitorio(s),3 Baño(s),170,104<br />\n¡Increíble oportunidad de inversión...,3865902,Piso,Madrid,NaN,0.0,0.0,0.0,2024-05-18,200.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Malasana,NaN,NaN,Malasana
3,https://www.pisosmadrid.com.es/propiedad/38659...,"Piso en venta en Pueblo Nuevo, Madrid",185.000 €,1,1 baño,75,"104<br />\nExcepcional local bajo tipo loft, t...",3865903,Piso,Madrid,NaN,0.0,0.0,0.0,2024-05-18,110.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Pueblo Nuevo,Pueblo Nuevo,Ciudad Lineal,NaN
4,https://www.pisosmadrid.com.es/propiedad/38663...,"Piso en venta en Villaverde, Madrid",86.000 €,3 Dormitorio(s),1 baño,60,"¡¡OPORTUNIDAD, IDEAL INVERSORES!! EL INMUEBLE ...",3866311,Piso,Madrid,NaN,0.0,0.0,0.0,2024-05-19,130.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Villaverde,NaN,Villaverde,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
889,https://www.pisosmadrid.com.es/propiedad/40864...,"Piso en venta en Los Angeles, Madrid",70.256 €,3 Dormitorio(s),1 baño,57,"Vivienda situada en Calle Godella,, en Madrid ...",4086495,Piso,Madrid,NaN,0.0,0.0,0.0,2024-10-24,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Los Angeles,NaN,NaN,Los Angeles
890,https://www.pisosmadrid.com.es/propiedad/40865...,"Piso en venta en Arganzuela, Madrid",740.000 €,4 Dormitorio(s),3 Baño(s),158,Se ofrece una magnífica oportunidad para adqui...,4086503,Piso,Madrid,NaN,0.0,0.0,0.0,2024-10-24,30.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,Arganzuela,NaN,Arganzuela,NaN
891,https://www.pisosmadrid.com.es/propiedad/40865...,Piso en venta en Madrid,329.000 €,3 Dormitorio(s),2 Baño(s),95,~SERVICHECK INMOBILIARIA comercializa un preci...,4086544,Piso,Madrid,NaN,0.0,1.0,1.0,2024-10-24,20.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,nan,NaN,NaN,nan
892,https://www.pisosmadrid.com.es/propiedad/40866...,"Piso en venta en Ibiza, Madrid",520.000 €,3 Dormitorio(s),1 baño,70,Precioso piso reformado de 60m2 enfrente del R...,4086665,Piso,Madrid,NaN,0.0,0.0,0.0,2024-10-24,40.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Ibiza,Ibiza,Retiro,NaN


In [120]:
dfpisosmadrid.shape

(894, 28)

In [121]:
dfpisosmadrid.columns

Index(['url_website', 'descripcion', 'precio', 'cant_dormitorios',
       'cant_banos', 'metros_cuadrados_construidos', 'descripcion_extendida',
       'referencia', 'tipo_lugar', 'ciudad', 'barrio/localidad/distrito',
       'piscina', 'garaje', 'jardin', 'fecha_publicacion', 'n.visitas',
       'calefaccion', 'trastero', 'amueblado', 'chimenea', 'lavadero',
       'videoportero', 'alarma', 'aire_acondicionado', 'area', 'barrios',
       'distrito', 'localidad'],
      dtype='object')

In [122]:
dfpisosmadrid.tail(1)

,url_website,descripcion,precio,cant_dormitorios,cant_banos,metros_cuadrados_construidos,descripcion_extendida,referencia,tipo_lugar,ciudad,barrio/localidad/distrito,piscina,garaje,jardin,fecha_publicacion,n.visitas,calefaccion,trastero,amueblado,chimenea,lavadero,videoportero,alarma,aire_acondicionado,area,barrios,distrito,localidad
893,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,nan,NaN,0.0,0.0,0.0,NaT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,nan,NaN,NaN,nan


In [123]:
#eliminar ultima fila
dfpisosmadrid= dfpisosmadrid.drop(dfpisosmadrid.index[-1])

In [124]:
# Función para normalizar las cadenas de texto
def normalize_string(s):
    if pd.notna(s):
        s = unicodedata.normalize('NFD', s)  # Descomponer los caracteres con acento
        s = ''.join([c for c in s if not unicodedata.combining(c)])  # Eliminar los caracteres de acento
        s = s.lower()  # Convertir a minúsculas
    return s

In [125]:
# Aqui filtramos barrio y distritos por una lista de columnas y por el excel donde estan guardados una cierta lista de barrios y distritos.
def clasificar_distrito_y_barrio_por_columnas(datos_pisosmadrid_detalles, dbcp_ordenado):
    # Crear las listas de barrios y distritos desde el dataframe de referencia
    distritos = dbcp_ordenado["nombre_distrito"].tolist()
    barrios = dbcp_ordenado["nombre_barrio"].tolist()

    # Recorrer las filas del dataframe
    for idx, fila in datos_pisosmadrid_detalles.iterrows():
        # Recorrer las tres columnas en orden de prioridad: descripcion, descripcion_extendida, url_website
        for columna in ['descripcion', 'descripcion_extendida', 'url_website']:
            valor = fila[columna]
            
            # Verificamos que el valor no sea nulo
            if pd.notna(valor):
                # Buscamos si el nombre del distrito aparece en la columna
                for distrito in distritos:
                    if distrito in valor and pd.isna(fila["distrito"]):
                        datos_pisosmadrid_detalles.at[idx, "distrito"] = distrito  # Asignar el nombre del distrito a la columna "distrito"

                # Buscamos si el nombre del barrio aparece en la columna
                for barrio in barrios:
                    if barrio in valor and pd.isna(fila["barrios"]):
                        datos_pisosmadrid_detalles.at[idx, "barrios"] = barrio  # Asignar el nombre del barrio a la columna "barrio"

    return datos_pisosmadrid_detalles




In [126]:
# Llamar a la función pasándole el dataframe y el dataframe de referencia
dfpisosmadrid = clasificar_distrito_y_barrio_por_columnas(dfpisosmadrid, dbcp_ordenado)

In [127]:
# Esta funcion rellena los distritos conociendo los barrios

def rellenar_distritos(datos_pisosmadrid_detalles,dbcp_ordenado):

    barrio_a_distrito=dict(zip(dbcp_ordenado["nombre_barrio"],dbcp_ordenado["nombre_distrito"]))

    for idx, fila in datos_pisosmadrid_detalles.iterrows():
        if pd.isna(fila["distrito"])and not pd.isna(fila["barrios"]):
            distrito = barrio_a_distrito.get(fila["barrios"])
            if distrito:
                datos_pisosmadrid_detalles.at[idx,"distrito"] = distrito 




In [128]:
rellenar_distritos(datos_pisosmadrid_detalles=dfpisosmadrid,dbcp_ordenado=dbcp_ordenado)

In [129]:

def clasificar_distrito_y_barrio_por_columnas(datos_pisosmadrid_detalles, dbcp_ordenado):
    # Crear las listas de barrios y distritos desde el dataframe de referencia
    distritos = dbcp_ordenado["nombre_distrito"].apply(lambda x: normalize_string(x)).tolist()
    barrios = dbcp_ordenado["nombre_barrio"].apply(lambda x: normalize_string(x)).tolist()

    # Recorrer las filas del dataframe
    for idx, fila in datos_pisosmadrid_detalles.iterrows():
        # Recorrer las tres columnas en orden de prioridad: descripcion, descripcion_extendida, url_website
        for columna in ['descripcion', 'descripcion_extendida', 'url_website',"localidad"]:
            valor = fila[columna]
            
            # Verificamos que el valor no sea nulo
            if pd.notna(valor):
                valor_normalizado = normalize_string(valor)
                
                # Verificamos si el valor normalizado contiene el nombre de un distrito o barrio
                for distrito, barrio in zip(distritos, barrios):
                    if distrito in valor_normalizado and pd.isna(fila["distrito"]):
                        datos_pisosmadrid_detalles.at[idx, "distrito"] = distrito  # Asignar el nombre del distrito a la columna "distrito"
                    if barrio in valor_normalizado and pd.isna(fila["barrios"]):
                        datos_pisosmadrid_detalles.at[idx, "barrios"] = barrio  # Asignar el nombre del barrio a la columna "barrio"

    return datos_pisosmadrid_detalles



In [130]:
# Llamar a la función pasándole el dataframe y el dataframe de referencia
dfpisosmadrid = clasificar_distrito_y_barrio_por_columnas(dfpisosmadrid, dbcp_ordenado)

In [131]:
# Esta funcion rellena los distritos conociendo los barrios al igual que antes pero ahora normalizamos los valores antes de rellenar para
# encontrar mas valores


def rellenar_distritos(datos_pisosmadrid_detalles, dbcp_ordenado):
    # Crear un diccionario de barrios a distritos normalizado
    barrio_a_distrito = {normalize_string(barrio): distrito for barrio, distrito in zip(dbcp_ordenado["nombre_barrio"], dbcp_ordenado["nombre_distrito"])}

    for idx, fila in datos_pisosmadrid_detalles.iterrows():
        # Verificamos si el valor de distrito está vacío y si hay un valor en barrios
        if pd.isna(fila["distrito"]) and not pd.isna(fila["barrios"]):
            barrio_normalizado = normalize_string(fila["barrios"])  # Normalizar el valor del barrio
            distrito = barrio_a_distrito.get(barrio_normalizado)  # Buscar el distrito usando el barrio normalizado
            if distrito:
                datos_pisosmadrid_detalles.at[idx, "distrito"] = distrito  # Asignar el distrito correspondiente



In [132]:
# Llamar a la función pasándole el dataframe y el dataframe de referencia
rellenar_distritos(datos_pisosmadrid_detalles=dfpisosmadrid, dbcp_ordenado=dbcp_ordenado)

In [133]:
# Aca vamos a hacer la filtracion desde la columna localidad y el dataframe de referencia  dbcp_ordenado donde hay valores de distritos y barrios



def clasificar_en_localidad(datos_pisosmadrid_detalles, dbcp_ordenado):
    # Crear las listas de barrios y distritos desde el dataframe de referencia
    distritos = dbcp_ordenado["nombre_distrito"].apply(lambda x: normalize_string(x)).tolist()
    barrios = dbcp_ordenado["nombre_barrio"].apply(lambda x: normalize_string(x)).tolist()

    # Recorrer las filas del dataframe
    for idx, fila in datos_pisosmadrid_detalles.iterrows():
        # Verificar que la columna 'localidad' no sea nula
        if pd.notna(fila["localidad"]):
            valor_localidad = normalize_string(fila["localidad"])

            # Buscar si el valor de la localidad coincide con algún distrito
            for distrito in distritos:
                if distrito in valor_localidad and pd.isna(fila["distrito"]):
                    datos_pisosmadrid_detalles.at[idx, "distrito"] = distrito  # Asignar el nombre del distrito a la columna "distrito"

            # Buscar si el valor de la localidad coincide con algún barrio
            for barrio in barrios:
                if barrio in valor_localidad and pd.isna(fila["barrios"]):
                    datos_pisosmadrid_detalles.at[idx, "barrios"] = barrio  # Asignar el nombre del barrio a la columna "barrio"
            
            # Convertir la columna 'localidad' en NaN después de asignar el barrio o distrito
            datos_pisosmadrid_detalles.at[idx, "localidad"] = pd.NA

    return datos_pisosmadrid_detalles


In [134]:

# Llamar a la función pasándole el dataframe
dfpisosmadrid = clasificar_en_localidad(dfpisosmadrid, dbcp_ordenado)

In [135]:
# Aca seguimos escarbando mas y aruñando  para conseguir mas barrios y distritos

# Filtrar las filas donde no hay valor en barrio, pero sí en distrito y localidad
filtro_1 = dfpisosmadrid[pd.isna(dfpisosmadrid["barrios"]) & pd.notna(dfpisosmadrid["distrito"]) & pd.notna(dfpisosmadrid["localidad"])]

# Filtrar las filas donde no hay valor en distrito, pero sí en barrio y localidad
filtro_2 = dfpisosmadrid[pd.isna(dfpisosmadrid["distrito"]) & pd.notna(dfpisosmadrid["barrios"]) & pd.notna(dfpisosmadrid["localidad"])]


In [136]:
dfpisosmadrid["distrito"].value_counts()

distrito
Centro                  134
Salamanca               118
Carabanchel              58
Arganzuela               43
Puente de Vallecas       35
Chamberí                 33
Retiro                   33
Chamartín                32
Latina                   32
Villaverde               30
Ciudad Lineal            29
 San Blas-Canillejas     27
Usera                    26
Moncloa-Aravaca          25
Tetuán                   20
Fuencarral-El Pardo      15
tetuan                   11
Hortaleza                10
centro                    9
 Villa de Vallecas        8
chamberi                  7
Moratalaz                 7
chamartin                 5
 vicalvaro                4
 Barajas                  4
 villa de vallecas        2
villaverde                2
ciudad lineal             1
carabanchel               1
 Vicálvaro                1
Name: count, dtype: int64

In [137]:
# Ahora vamos a filtrar por un diccionario que creamos abajo y tambien la columna localidad


# Función que recorre la columna localidad y asigna los barrios correspondientes
def asignar_barrios_a_localidad(datos_pisosmadrid_detalles, diccionario_barrios):
    # Normalizar los barrios en el diccionario
    diccionario_normalizado = {
        normalize_string(barrio): distrito for distrito, barrios in diccionario_barrios.items() for barrio in barrios
    }
    
    # Recorrer el dataframe
    for idx, fila in datos_pisosmadrid_detalles.iterrows():
        # Solo proceder si la columna localidad no es nula
        localidad = fila['localidad']
        if pd.notna(localidad):
            localidad_normalizada = normalize_string(localidad)

            # Comprobar si el valor normalizado de localidad contiene algún barrio del diccionario
            for barrio_normalizado, distrito in diccionario_normalizado.items():
                if barrio_normalizado in localidad_normalizada and pd.isna(fila['barrios']):
                    # Asignar el barrio en la columna "barrios"
                    datos_pisosmadrid_detalles.at[idx, 'barrios'] = barrio_normalizado

    return datos_pisosmadrid_detalles

# Diccionario de ejemplo con distritos y barrios (ya normalizados)
diccionario_barrios = {
    "Centro": ["Malasaña", "La Latina", "Chueca","malasana"],
    "Moratalaz": ["La Elipa"],
    "Moncloa-Aravaca": ["Puerta de Hierro", "Príncipe Pío"],
    "Fuencarral-El Pardo": ["Buena Vista", "Ventilla", "Fuencarral"],
    "Chamartín": ["Bernabéu"],
    "Ciudad Lineal": ["Arturo Soria"],
    "San Blas-Canillejas": ["Las Mercedes","las mercedes"],
    "Villaverde": ["Villaverde Alto", "Villaverde Bajo"],
    "Carabanchel": ["Pau Carabanchel"],
}



In [138]:
# Llamar a la función pasándole el dataframe y el diccionario de barrios
dfpisosmadrid= asignar_barrios_a_localidad(dfpisosmadrid, diccionario_barrios)

**_La funcion de arriba no genero cambio  pero ahora se que no tengo que hacerla mas adelante, asi que no la eliminare..... dejo este nota aqui_**

In [139]:

# Aca vamos a hacer filtraciones como antes pero con un par de condicionantes mas... Escarbando tanto como den las ideas.


# Función que recorre la columna localidad y asigna los barrios correspondientes
def asignar_barrios_y_distritos(datos_pisosmadrid_detalles, diccionario_barrios):
    # Normalizar los barrios y distritos en el diccionario
    diccionario_normalizado = {
        normalize_string(barrio): distrito for distrito, barrios in diccionario_barrios.items() for barrio in barrios
    }
    
    # Recorrer el dataframe
    for idx, fila in datos_pisosmadrid_detalles.iterrows():
        localidad = fila['localidad']
        
        # Comprobar si existe un valor en localidad y no hay valores en barrios ni distrito
        if pd.notna(localidad) and pd.isna(fila['barrios']) and pd.isna(fila['distrito']):
            localidad_normalizada = normalize_string(localidad)
            
            # Comprobar si el valor normalizado de localidad contiene algún barrio del diccionario
            for barrio_normalizado, distrito in diccionario_normalizado.items():
                if barrio_normalizado in localidad_normalizada:
                    datos_pisosmadrid_detalles.at[idx, 'barrios'] = barrio_normalizado  # Asignar barrio a la columna "barrios"
                    datos_pisosmadrid_detalles.at[idx, 'distrito'] = distrito  # Asignar distrito correspondiente

            # Si no se encuentra un barrio, intentar asignar un distrito (buscando el nombre del distrito)
            if pd.isna(fila['barrios']):
                for distrito in diccionario_barrios.keys():
                    if normalize_string(distrito) in localidad_normalizada:
                        datos_pisosmadrid_detalles.at[idx, 'distrito'] = distrito  # Asignar el distrito si no se encontró barrio
                        
    return datos_pisosmadrid_detalles

# Diccionario de ejemplo con distritos y barrios (ya normalizados)
diccionario_barrios = {
    "Centro": ["Malasaña", "La Latina", "Chueca","malasana"],
    "Moratalaz": ["La Elipa"],
    "Moncloa-Aravaca": ["Puerta de Hierro", "Príncipe Pío"],
    "Fuencarral-El Pardo": ["Buena Vista", "Ventilla", "Fuencarral"],
    "Chamartín": ["Bernabéu"],
    "Ciudad Lineal": ["Arturo Soria"],
    "San Blas-Canillejas": ["Las Mercedes","las mercedes"],
    "Villaverde": ["Villaverde Alto", "Villaverde Bajo"],
    "Carabanchel": ["Pau Carabanchel"],
    
}



In [140]:
# Llamar a la función pasándole el dataframe y el diccionario de barrios
dfpisosmadrid= asignar_barrios_y_distritos(dfpisosmadrid, diccionario_barrios)

In [141]:
dfpisosmadrid["distrito"].isna().sum()

131

In [142]:
dfpisosmadrid["barrios"].isna().sum()

282

In [143]:
dfpisosmadrid.shape

(893, 28)

In [144]:
barrio_distrito_info= {
    "Centro": ["Malasaña", "La Latina", "Chueca","malasana"],
    "Moratalaz": ["La Elipa"],
    "Moncloa-Aravaca": ["Puerta de Hierro", "Príncipe Pío"],
    "Fuencarral-El Pardo": ["Buena Vista", "Ventilla", "Fuencarral"],
    "Chamartín": ["Bernabéu"],
    "Ciudad Lineal": ["Arturo Soria"],
    "San Blas-Canillejas": ["Las Mercedes","las mercedes"],
    "Villaverde": ["Villaverde Alto", "Villaverde Bajo"],
    "Carabanchel": ["Pau Carabanchel"],
    
}


def corregir_distrito_por_barrio(datos_pisosmadrid_detalles, dbcp_ordenado, barrio_distrito_info):
    # Recorrer las filas del dataframe original
    filas_sin_coincidencias = []
    
    for idx, fila in datos_pisosmadrid_detalles.iterrows():
        barrio = fila["barrios"]
        distrito_actual = fila["distrito"]
        
        if pd.notna(barrio):
            barrio_normalizado = normalize_string(barrio)
            
            # Buscar la coincidencia en el dataframe de referencia
            distrito_en_ref = dbcp_ordenado[dbcp_ordenado["nombre_barrio"].apply(lambda x: normalize_string(x) == barrio_normalizado)]
            
            if not distrito_en_ref.empty:
                # Si hay coincidencia en el dataframe de referencia
                distrito_nuevo = distrito_en_ref.iloc[0]["nombre_distrito"]
                
                # Asignar el nuevo distrito si es diferente al actual
                if pd.isna(distrito_actual) or distrito_actual != distrito_nuevo:
                    datos_pisosmadrid_detalles.at[idx, "distrito"] = distrito_nuevo
            else:
                # Si no se encuentra en el dataframe de referencia, buscar en el diccionario
                for (distrito, barrio_ref), (cod_distrito, cod_barrio, cod_barrio2, codigo_postal) in barrio_distrito_info.items():
                    if normalize_string(barrio_ref) == barrio_normalizado:
                        # Asignar el distrito desde el diccionario
                        if pd.isna(distrito_actual) or distrito_actual != distrito:
                            datos_pisosmadrid_detalles.at[idx, "distrito"] = distrito
                        break
                else:
                    # Si no se encuentra en el dataframe de referencia ni en el diccionario, mostrar la fila
                    filas_sin_coincidencias.append(fila)
    
    return datos_pisosmadrid_detalles, pd.DataFrame(filas_sin_coincidencias)



In [145]:
# Llamar a la función
datos_actualizados, filas_sin_coincidencias = corregir_distrito_por_barrio(dfpisosmadrid, dbcp_ordenado, barrio_distrito_info)

# Mostrar las filas que no tienen coincidencias
filas_sin_coincidencias

""


In [146]:
# Aqui eliminamos dos columnas con informacion sin utilidad
columnas_baja= ["ciudad","barrio/localidad/distrito",]

dfpisosmadrid= dfpisosmadrid.drop(columns=columnas_baja)

In [147]:
barrio_distrito_info= {
    "Centro": ["Malasaña", "La Latina", "Chueca","malasana"],
    "Moratalaz": ["La Elipa"],
    "Moncloa-Aravaca": ["Puerta de Hierro", "Príncipe Pío"],
    "Fuencarral-El Pardo": ["Buena Vista", "Ventilla", "Fuencarral"],
    "Chamartín": ["Bernabéu"],
    "Ciudad Lineal": ["Arturo Soria"],
    "San Blas-Canillejas": ["Las Mercedes","las mercedes"],
    "Villaverde": ["Villaverde Alto", "Villaverde Bajo"],
    "Carabanchel": ["Pau Carabanchel"],
    
}

In [148]:
# Aqui agrego los datos de cod_ditrito al dataframe original.Normalizando




    # Asigna el valor de cod_distrito en el dataframe original basado en el dataframe de referencia o sea el dbcp_ordenado
def asignar_cod_distrito(datos_pisosmadrid_detalles, dbcp_ordenado):

    # Asegurarse de que la columna cod_distrito exista en el dataframe original
    if "cod_distrito" not in datos_pisosmadrid_detalles.columns:
        datos_pisosmadrid_detalles["cod_distrito"] = pd.NA

    # Normalizar la columna nombre_distrito del dataframe de referencia
    dbcp_ordenado["nombre_distrito_normalizado"] = dbcp_ordenado["nombre_distrito"].apply(normalize_string)

    # Inicializar una lista para las filas sin coincidencias
    filas_sin_coincidencia = []

    # Recorrer el dataframe original
    for idx, fila in datos_pisosmadrid_detalles.iterrows():
        distrito = fila["distrito"]

        if pd.notna(distrito):
            distrito_normalizado = normalize_string(distrito)

            # Buscar coincidencia en el dataframe de referencia
            coincidencia = dbcp_ordenado[dbcp_ordenado["nombre_distrito_normalizado"] == distrito_normalizado]

            if not coincidencia.empty:
                # Si hay coincidencia, asignar el valor de cod_distrito
                datos_pisosmadrid_detalles.at[idx, "cod_distrito"] = coincidencia.iloc[0]["cod_distrito"]
            else:
                # Si no hay coincidencia, agregar la fila a las filas sin coincidencias
                filas_sin_coincidencia.append(fila)

    # Crear un DataFrame con las filas sin coincidencias
    distrito_sin_coincidencia = pd.DataFrame(filas_sin_coincidencia)

    return datos_pisosmadrid_detalles, distrito_sin_coincidencia



In [149]:

# Ejecución de la función
datos_actualizados, distrito_sin_coincidencia = asignar_cod_distrito(dfpisosmadrid, dbcp_ordenado)

In [150]:
filas_sin_coincidencias

""


In [151]:
# Aca le asigno los valores a las columnas cod_barrio y cod_barrio_2 usando el dataframe de referencia 

def asignar_cod_barrios(datos_pisosmadrid_detalles, dbcp_ordenado):

    # Asegurarse de que las columnas cod_barrio y cod_barrio_2 existan en el dataframe original
    if "cod_barrio" not in datos_pisosmadrid_detalles.columns:
        datos_pisosmadrid_detalles["cod_barrio"] = pd.NA
    if "cod_barrio_2" not in datos_pisosmadrid_detalles.columns:
        datos_pisosmadrid_detalles["cod_barrio_2"] = pd.NA

    # Normalizar la columna nombre_barrio del dataframe de referencia
    dbcp_ordenado["nombre_barrio_normalizado"] = dbcp_ordenado["nombre_barrio"].apply(normalize_string)

    # Inicializar una lista para las filas sin coincidencias
    filas_sin_coincidencia = []

    # Recorrer el dataframe original
    for idx, fila in datos_pisosmadrid_detalles.iterrows():
        barrio = fila["barrios"]

        if pd.notna(barrio):
            barrio_normalizado = normalize_string(barrio)

            # Buscar coincidencia en el dataframe de referencia
            coincidencia = dbcp_ordenado[dbcp_ordenado["nombre_barrio_normalizado"] == barrio_normalizado]

            if not coincidencia.empty:
                # Si hay coincidencia, asignar los valores de cod_barrio y cod_barrio_2
                datos_pisosmadrid_detalles.at[idx, "cod_barrio"] = coincidencia.iloc[0]["cod_barrio"]
                datos_pisosmadrid_detalles.at[idx, "cod_barrio_2"] = coincidencia.iloc[0]["cod_barrio_2"]
            else:
                # Si no hay coincidencia, agregar la fila a las filas sin coincidencias
                filas_sin_coincidencia.append(fila)

    # Crear un DataFrame con las filas sin coincidencias
    barrio_sin_coincidencia = pd.DataFrame(filas_sin_coincidencia)

    return datos_pisosmadrid_detalles, barrio_sin_coincidencia


In [152]:

# Ejecución de la función
datos_actualizados, barrio_sin_coincidencia = asignar_cod_barrios(dfpisosmadrid, dbcp_ordenado)

In [153]:
# Aca voy a asignar los codigos postales tomando en cuenta las columnas barrio y nombre_barrio

def asignar_cod_barrios(datos_pisosmadrid_detalles, dbcp_ordenado):
    
    # Asegurarse de que la columna codigo_Postal exista en el dataframe original
    if "codigo_Postal" not in datos_pisosmadrid_detalles.columns:
        datos_pisosmadrid_detalles["codigo_Postal"] = pd.NA

    # Normalizar la columna barrios del dataframe original
    datos_pisosmadrid_detalles["barrios_normalizado"] = datos_pisosmadrid_detalles["barrios"].apply(normalize_string)

    # Normalizar la columna nombre_barrio del dataframe de referencia
    dbcp_ordenado["nombre_barrio_normalizado"] = dbcp_ordenado["nombre_barrio"].apply(normalize_string)

    # Inicializar una lista para las filas sin coincidencias
    filas_sin_coincidencia = []

    # Recorrer el dataframe original
    for idx, fila in datos_pisosmadrid_detalles.iterrows():
        barrio_normalizado = fila["barrios_normalizado"]

        if pd.notna(barrio_normalizado):
            # Buscar coincidencia en el dataframe de referencia usando la columna normalizada
            coincidencia = dbcp_ordenado[dbcp_ordenado["nombre_barrio_normalizado"] == barrio_normalizado]

            if not coincidencia.empty:
                # Si hay coincidencia, asignar el valor de `codigo_Postal`
                datos_pisosmadrid_detalles.at[idx, "codigo_Postal"] = coincidencia.iloc[0]["codigo_Postal"]
            else:
                # Si no hay coincidencia, agregar la fila a las filas sin coincidencias
                filas_sin_coincidencia.append(fila)

    # Crear un DataFrame con las filas sin coincidencias
    barrio_sin_coincidencia = pd.DataFrame(filas_sin_coincidencia)

    return datos_pisosmadrid_detalles, barrio_sin_coincidencia



In [154]:
# Ejecución de la función
datos_actualizados, barrio_sin_coincidencia = asignar_cod_barrios(dfpisosmadrid, dbcp_ordenado)

In [155]:

barrio_sin_coincidencia

""


In [156]:
dfpisosmadrid["cod_barrio_2"].isna().sum()

282

In [157]:
dfpisosmadrid["barrios"].isna().sum()

282

In [158]:
dfpisosmadrid= dfpisosmadrid.drop(columns=["barrios_normalizado","localidad"])

In [159]:
dfpisosmadrid.shape

(893, 29)

In [160]:
dfpisosmadrid.duplicated().sum()

0

In [161]:
dfpisosmadrid.isna().sum()

url_website                       0
descripcion                       0
precio                            0
cant_dormitorios                  0
cant_banos                        0
metros_cuadrados_construidos      0
descripcion_extendida             0
referencia                        0
tipo_lugar                        0
piscina                          11
garaje                           11
jardin                           11
fecha_publicacion                11
n.visitas                        11
calefaccion                      11
trastero                         11
amueblado                        11
chimenea                         11
lavadero                         11
videoportero                     11
alarma                           11
aire_acondicionado               11
area                             11
barrios                         282
distrito                        131
cod_distrito                    131
cod_barrio                      282
cod_barrio_2                

In [162]:
pisos_madrid_unido=dfpisosmadrid

In [163]:
pisos_madrid_unido.columns.duplicated().sum()

0

In [164]:
pisos_madrid_unido.shape

(893, 29)

In [167]:
pisos_madrid_unido.query("distrito.isnull()")

,url_website,descripcion,precio,cant_dormitorios,cant_banos,metros_cuadrados_construidos,descripcion_extendida,referencia,tipo_lugar,piscina,garaje,jardin,fecha_publicacion,n.visitas,calefaccion,trastero,amueblado,chimenea,lavadero,videoportero,alarma,aire_acondicionado,area,barrios,distrito,cod_distrito,cod_barrio,cod_barrio_2,codigo_Postal
0,https://www.pisosmadrid.com.es/propiedad/38638...,Piso en venta en Madrid,60.000 €,3 Dormitorio(s),1 baño,54,PISO EN MADRID. ~Vivienda situada en Avenida P...,3863841,Piso,0.0,0.0,0.0,2024-05-17,120.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,nan,NaN,NaN,<NA>,<NA>,<NA>,<NA>
5,https://www.pisosmadrid.com.es/propiedad/38666...,"Apartamento en venta en Malasaña, Madrid",839.000 €,3 Dormitorio(s),3 Baño(s),110,*Inmueble Reformado en Calle San Bernardo*~~De...,3866604,Apartamento,0.0,0.0,0.0,2024-05-19,130.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Malasana,NaN,NaN,<NA>,<NA>,<NA>,<NA>
11,https://www.pisosmadrid.com.es/propiedad/38671...,Piso en venta en Madrid,68.800 €,3 Dormitorio(s),1 baño,60,"Piso 3 dormitorios, 1 baños, 0 garajes, Para r...",3867131,Piso,0.0,0.0,0.0,2024-05-20,30.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,nan,NaN,NaN,<NA>,<NA>,<NA>,<NA>
27,https://www.pisosmadrid.com.es/propiedad/38722...,Piso en venta en Madrid,195.000 €,3 Dormitorio(s),2 Baño(s),102,DP2020 vende piso en edificio residencial con ...,3872239,Piso,0.0,0.0,0.0,2024-05-23,140.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,nan,NaN,NaN,<NA>,<NA>,<NA>,<NA>
29,https://www.pisosmadrid.com.es/propiedad/38742...,Piso en venta en Madrid,130.000 €,2 Dormitorio(s),1 baño,59,PISO EN MADRID. ~Vivienda situada en la Calle ...,3874208,Piso,0.0,0.0,0.0,2024-05-24,90.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,nan,NaN,NaN,<NA>,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
858,https://www.pisosmadrid.com.es/propiedad/40828...,Piso en venta en Madrid,285.000 €,2 Dormitorio(s),1 baño,70,Inmobiliarias Encuentro Vende Piso listo para ...,4082845,Piso,0.0,1.0,0.0,2024-10-22,40.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,nan,NaN,NaN,<NA>,<NA>,<NA>,<NA>
877,https://www.pisosmadrid.com.es/propiedad/40851...,Apartamento en venta en Madrid,315.000 €,0 Dormitorio(s),1 baño,71,Te presentamos este fantástico estudio en la c...,4085127,Apartamento,1.0,0.0,0.0,2024-10-24,20.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,nan,NaN,NaN,<NA>,<NA>,<NA>,<NA>
886,https://www.pisosmadrid.com.es/propiedad/40863...,"Piso en venta en Malasaña, Madrid",1.400.000 €,5 Dormitorio(s),2 Baño(s),212,¡Gran Oportunidad en Malasaña! Vivienda amplia...,4086303,Piso,0.0,0.0,0.0,2024-10-24,20.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Malasana,NaN,NaN,<NA>,<NA>,<NA>,<NA>
888,https://www.pisosmadrid.com.es/propiedad/40864...,Piso en venta en Madrid,97.920 €,2 Dormitorio(s),1 baño,50,"Vivienda situada en Calle Transmonte,, en Madr...",4086493,Piso,0.0,0.0,0.0,2024-10-24,30.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,nan,NaN,NaN,<NA>,<NA>,<NA>,<NA>


In [168]:
pisos_madrid_unido

,url_website,descripcion,precio,cant_dormitorios,cant_banos,metros_cuadrados_construidos,descripcion_extendida,referencia,tipo_lugar,piscina,garaje,jardin,fecha_publicacion,n.visitas,calefaccion,trastero,amueblado,chimenea,lavadero,videoportero,alarma,aire_acondicionado,area,barrios,distrito,cod_distrito,cod_barrio,cod_barrio_2,codigo_Postal
0,https://www.pisosmadrid.com.es/propiedad/38638...,Piso en venta en Madrid,60.000 €,3 Dormitorio(s),1 baño,54,PISO EN MADRID. ~Vivienda situada en Avenida P...,3863841,Piso,0.0,0.0,0.0,2024-05-17,120.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,nan,NaN,NaN,<NA>,<NA>,<NA>,<NA>
1,https://www.pisosmadrid.com.es/propiedad/38658...,"Piso en venta en Montecarmelo - El Goloso, Madrid",825.000 €,4 Dormitorio(s),2 Baño(s),142,600<br />\nAprovecha esta oportunidad de compr...,3865872,Piso,1.0,1.0,1.0,2024-05-18,150.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,Montecarmelo El Goloso,El Goloso,Fuencarral-El Pardo,8,88,8,28049
2,https://www.pisosmadrid.com.es/propiedad/38659...,"Piso en venta en Malasaña, Madrid",760.000 €,5 Dormitorio(s),3 Baño(s),170,104<br />\n¡Increíble oportunidad de inversión...,3865902,Piso,0.0,0.0,0.0,2024-05-18,200.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Malasana,NaN,centro,1,<NA>,<NA>,<NA>
3,https://www.pisosmadrid.com.es/propiedad/38659...,"Piso en venta en Pueblo Nuevo, Madrid",185.000 €,1,1 baño,75,"104<br />\nExcepcional local bajo tipo loft, t...",3865903,Piso,0.0,0.0,0.0,2024-05-18,110.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Pueblo Nuevo,Pueblo Nuevo,Ciudad Lineal,15,152,2,28017
4,https://www.pisosmadrid.com.es/propiedad/38663...,"Piso en venta en Villaverde, Madrid",86.000 €,3 Dormitorio(s),1 baño,60,"¡¡OPORTUNIDAD, IDEAL INVERSORES!! EL INMUEBLE ...",3866311,Piso,0.0,0.0,0.0,2024-05-19,130.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Villaverde,NaN,Villaverde,17,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
888,https://www.pisosmadrid.com.es/propiedad/40864...,Piso en venta en Madrid,97.920 €,2 Dormitorio(s),1 baño,50,"Vivienda situada en Calle Transmonte,, en Madr...",4086493,Piso,0.0,0.0,0.0,2024-10-24,30.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,nan,NaN,NaN,<NA>,<NA>,<NA>,<NA>
889,https://www.pisosmadrid.com.es/propiedad/40864...,"Piso en venta en Los Angeles, Madrid",70.256 €,3 Dormitorio(s),1 baño,57,"Vivienda situada en Calle Godella,, en Madrid ...",4086495,Piso,0.0,0.0,0.0,2024-10-24,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Los Angeles,los angeles,Villaverde,17,174,4,28021
890,https://www.pisosmadrid.com.es/propiedad/40865...,"Piso en venta en Arganzuela, Madrid",740.000 €,4 Dormitorio(s),3 Baño(s),158,Se ofrece una magnífica oportunidad para adqui...,4086503,Piso,0.0,0.0,0.0,2024-10-24,30.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,Arganzuela,Delicias,Arganzuela,2,25,5,28045
891,https://www.pisosmadrid.com.es/propiedad/40865...,Piso en venta en Madrid,329.000 €,3 Dormitorio(s),2 Baño(s),95,~SERVICHECK INMOBILIARIA comercializa un preci...,4086544,Piso,0.0,1.0,1.0,2024-10-24,20.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,nan,NaN,NaN,<NA>,<NA>,<NA>,<NA>


In [169]:
pisos_madrid_unido.shape

(893, 29)

In [170]:
pisos_madrid_unido.isna().sum()

url_website                       0
descripcion                       0
precio                            0
cant_dormitorios                  0
cant_banos                        0
metros_cuadrados_construidos      0
descripcion_extendida             0
referencia                        0
tipo_lugar                        0
piscina                          11
garaje                           11
jardin                           11
fecha_publicacion                11
n.visitas                        11
calefaccion                      11
trastero                         11
amueblado                        11
chimenea                         11
lavadero                         11
videoportero                     11
alarma                           11
aire_acondicionado               11
area                             11
barrios                         282
distrito                        131
cod_distrito                    131
cod_barrio                      282
cod_barrio_2                

In [171]:
pd.set_option('display.max_colwidth', None)

In [172]:
pisos_madrid_unido.query("lavadero==1")[["url_website","descripcion_extendida"]]

,url_website,descripcion_extendida
72,https://www.pisosmadrid.com.es/propiedad/3895070-piso-en-venta-en-salamanca-madrid,"GUINDALERA-CALLE MARTÃNEZ IZQUIERDO. PLANTA BAJA. Piso reciÃ©n reformado,Â con mucha luz, con ventilaciÃ³n natural en todas las estancias, 2Â Dormitorios dobles, uno con vestidor, 2 baÃ±os, armarios empotrados. La cocin..."


In [173]:
pisos_madrid_unido=pisos_madrid_unido.drop(columns="lavadero")

In [174]:
pisos_madrid_unido['web']="pisosmadrid"
pisos_madrid_unido['fecha_descarga']="24/10/2024"

In [175]:
pisos_madrid_unido.columns

Index(['url_website', 'descripcion', 'precio', 'cant_dormitorios',
       'cant_banos', 'metros_cuadrados_construidos', 'descripcion_extendida',
       'referencia', 'tipo_lugar', 'piscina', 'garaje', 'jardin',
       'fecha_publicacion', 'n.visitas', 'calefaccion', 'trastero',
       'amueblado', 'chimenea', 'videoportero', 'alarma', 'aire_acondicionado',
       'area', 'barrios', 'distrito', 'cod_distrito', 'cod_barrio',
       'cod_barrio_2', 'codigo_Postal', 'web', 'fecha_descarga'],
      dtype='object')

In [176]:
# Diccionario con los nuevos nombres de las columnas
#voy a suponer que la superficie es superficie construida
nuevos_nombres ={'url_website':'url','descripcion': 'title',
'metros_cuadrados_construidos': 'm2_constr',
'cant_dormitorios': 'dormitorios',
'cant_banos': 'banos' ,
'tipo_lugar':'tipologia',
'area':'zona',
'barrios':'barrio'
} 
pisos_madrid_unido = pisos_madrid_unido.rename(columns=nuevos_nombres)

In [180]:
pisos_madrid_unido.query("url=='https://www.pisosmadrid.com.es/propiedad/3891215-piso-en-venta-en-madrid'")

,url,title,precio,dormitorios,banos,m2_constr,descripcion_extendida,referencia,tipologia,piscina,garaje,jardin,fecha_publicacion,n.visitas,calefaccion,trastero,amueblado,chimenea,videoportero,alarma,aire_acondicionado,zona,barrio,distrito,cod_distrito,cod_barrio,cod_barrio_2,codigo_Postal,web,fecha_descarga
70,https://www.pisosmadrid.com.es/propiedad/3891215-piso-en-venta-en-madrid,Piso en venta en Madrid,126.000 €,1,1 baño,52.75,"Tengo Piso Arcentales comercializa en exclusiva coqueta vivienda en el reconocido barrio de Amposta. ~La finca se compone de: un dormitorio doble, una confortable cocina abierta al salón - comedor lo que le aporta una s...",3891215,Piso,0.0,0.0,0.0,2024-06-10,120.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,nan,Amposta,San Blas-Canillejas,20,203,3,28037,pisosmadrid,24/10/2024


In [181]:
pisos_madrid_unido.to_csv("dfpisosmadrid_paraunir.csv",index="False")